# 第一題 #
讀取2017所有表特版的文章

In [1]:
import requests
import datetime
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import codecs
import time
END_POST = '[正妹] 新年 第889彈'

# function part #

* check_date_from_page
傳進頁面的soup，檢查是否存在12/31號的文章
若有，回傳該文章之soup

* get_website
傳入url，回傳該網站的網站位置

In [2]:
def check_date_from_page(cur_article):
    def check_date(soup):
        """
        input beautifuy object, check date
        must find last
        """
        # get date
        date_string = soup.find('div', 'date').string.strip()
        date = datetime.datetime.strptime(date_string, "%m/%d")
        if date.month != 12 or date.day != 31:
            return False
        else:
            return True

    cur_article = cur_article.find('div', 'r-ent')
    while cur_article is not None:
        if not check_date(cur_article):
            # next
            cur_article = cur_article.find_next('div', 'r-ent')
        else:
            # find last match
            while cur_article is not None:
                prev_article = cur_article
                cur_article = cur_article.find_next('div', 'r-ent')
                # got 1/1
                if not check_date(cur_article):
                    return prev_article
            return prev_article
    return False


def get_website(url):
    """
    get the website from url
    """
    site = urlparse(url)
    site_netloc = site.scheme + '://' + site.netloc
    return site_netloc


def get_response(url):
    response = requests.get(url)
    while not response.ok:
        response = requests.get(url)
    return response


def get_prev_page_url(root_soup):
    return website + root_soup.find('div', id='action-bar-container').find('div', 'btn-group-paging').find_all('a')[1]['href']

* 沒網址的不用爬，可忽略
* 爬2017年的所有文章
* 忽略分類為公告的文章
* output:
* 1. 所有文章 
  2. 所有爆文
* 檔案內容 - 日期,標題,url（無空格

## 結束條件 ##
https://www.ptt.cc/bbs/Beauty/M.1483208018.A.411.html

# main function start #

In [3]:
t0 = time.time()
# find the 12/31 article
url = "https://www.ptt.cc/bbs/Beauty/index.html"
# get ptt's website
website = get_website(url)
# first flag
first_flag = True
############### loading ####################
response = get_response(url)
root_soup = BeautifulSoup(response.text, 'lxml')
############### end loading ################
find_soup = check_date_from_page(root_soup)

while not find_soup:
    # find next page
    print('page: ' + url + ' not find!')
    print('get next page...')
    # prev page's url
    url = get_prev_page_url(root_soup)

    # if this is prist page, than direct minus 300 ...
    if first_flag == True:
        print("first page, direct jump ...")
        # got the number part
        match = re.search(r'(\d+)(?:.html$)', url)
        number = match[1]
        number = int(number) - 300
        # replace back
        url = re.sub(r"(\d+)(.html$)", str(number) + r'\2', url)
        print('jump to: ', url)
        first_flag = False

    # loading
    response = get_response(url)
    root_soup = BeautifulSoup(response.text, 'lxml')
    find_soup = check_date_from_page(root_soup)

print('find!')

# record time
t1 = time.time()
time_file = codecs.open("exec_time.txt", "a", "utf-8")
time_file.write(
    'total time for get the proper 2017 last post: ' + str(t1 - t0) + '\n')
time_file.close()

page: https://www.ptt.cc/bbs/Beauty/index.html not find!
get next page...
first page, direct jump ...
jump to:  https://www.ptt.cc/bbs/Beauty/index2360.html
page: https://www.ptt.cc/bbs/Beauty/index2360.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2359.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2358.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2357.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2356.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2355.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2354.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2353.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2352.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2351.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2350.html not find!
ge

# function for next part of code #


In [8]:
def get_date(soup):
    date_string = soup.find('div', 'date').string.strip()
    return date_string


def get_title(cur_soup):
    if cur_soup.find('div', 'title').find('a'):
        title = ''
        for string in cur_soup.find('div', 'title').a.strings:
            title = title + string
            # if @ occur, need specical parse
            # if string == '[email protected]':
            #     title = title + '@' + string['data-cfemail']
            # else:
            #     title = title + string
        return title
    else:
        return False


def write_info(all_file, pop_file, soup):
    """soup is on <div r-ent>"""
    def check_title_string(title):
        """title is a string, true mean this is what we need"""
        match = re.search(r'\[公告\]', title)
        if not match:
            return True
        else:
            return False

    # check if title.a exist
    if not soup.find('div', 'title').find('a'):
        return False
    # check title contain [公告] or not
    title = get_title(soup)
    if check_title_string(title):
        # grab date, topic, url
        date_temp = soup.find('div', 'date').string.strip()
        # remove /
        date = re.sub(r'[/]', '', date_temp)
        # title
        url = website + soup.find('div', 'title').find('a')['href']
        # merge to one line
        write_line = ','.join([date, title, url])
        # if 爆, write to pop_file
        thumb_number = soup.find('div', 'nrec')
        if thumb_number.find('span'):
            thumb_number = thumb_number.span.string.strip()
        else:
            tuhmb_number = ''
        if thumb_number == '爆':
            pop_file.write(write_line + '\n')
        all_file.write(write_line + '\n')
        # write to all_file

In [5]:
t2 = time.time()
# start soup
# root_soup is current soup on root
start_soup = find_soup
# open file
all_file = codecs.open("all_articles.txt", "w", "utf-8")
pop_file = codecs.open("all_popular.txt", "w", "utf-8")

# first page is special case
write_info(all_file, pop_file, start_soup)
for cur_soup in start_soup.find_all_previous('div', 'r-ent'):
    if get_title(cur_soup):
        print('write file for "' + get_title(cur_soup) + '"')
    write_info(all_file, pop_file, cur_soup)

# second page start
is_continued = True
while is_continued:
    # to next page
    url = get_prev_page_url(root_soup)
    response = get_response(url)
    root_soup = BeautifulSoup(response.text, 'lxml')

    # loop root_soup
    for cur_soup in reversed(root_soup.find_all('div', 'r-ent')):
        title = get_title(cur_soup)
        if title:
            date = get_date(cur_soup)
            print('write file for "' + date + " " + title + '"')
        write_info(all_file, pop_file, cur_soup)
        # ending point

        if title == END_POST:
            is_continued = False
            break
    is_continued == False
all_file.close()
pop_file.close()

# record time
t3 = time.time()
time_file = codecs.open("exec_time.txt", "a", "utf-8")
time_file.write('total time for crawl all 2017 post: ' +
                str(t3 - t2) + '\n')
time_file.write('total exec time: ' + str(t3 - t0) + '\n')

time_file.close()

write file for "[正妹] 2018 世界新車大展 Day 1"
write file for "[正妹] 2018"
write file for "[正妹] 奶特"
write file for "[正妹] 韓流"
write file for "[正妹] 彩彩小萌獸"
write file for "[正妹] 柏木由紀"
write file for "[正妹] 俄羅斯暗黑女模Helga Lovekaty"
write file for "[正妹] 彩美旬果"
write file for "[正妹] 秀氣長髮 "
write file for "[神人] 台北車展Polo beats DJ"
write file for "Re: [神人] hotels.com廣告女主角"
write file for "12/31 [正妹] 台北車展lexus正妹"
write file for "12/31 [正妹] 韓流女團藝人"
write file for "12/31 [神人] 自問自答"
write file for "12/31 [神人]台北車展land rover的sg"
write file for "12/31 [正妹] 跨年前發一波"
write file for "12/31 [帥哥] 最近好像流行這種"
write file for "12/31 [正妹] 企業排球聯賽 中纖美女主攻手"
write file for "12/30 [正妹] 無題"
write file for "12/30 [正妹] 世界第衣美"
write file for "12/30 [正妹] 高畑充希 Takahata Mitsuki"
write file for "12/30 [正妹] 兩個高三生"
write file for "12/30 [神人] 柯批剪綵旁的正妹"
write file for "12/30 [正妹] Madison Iseman"
write file for "12/30 [正妹] 李元玲"
write file for "12/30 [帥哥] 正太控請進"
write file for "12/30 [正妹] 浜辺美波"
write file for "12/30 [正妹] IU"
write file for "12/3

write file for "12/20 [神人] 台北馬拉松citizen的showgirl"
write file for "12/20 [正妹] 台灣海洋大學"
write file for "12/20 Re: [正妹] 筧美和子"
write file for "12/20 [正妹] 覺得不錯"
write file for "12/20 [神人] 福斯show girl"
write file for "12/20 [正妹] 筧美和子"
write file for "12/20 [神人] 一張圖求神"
write file for "12/20 [正妹] 神七"
write file for "12/20 [正妹] 馬友蓉?????"
write file for "12/20 [正妹] 車模"
write file for "12/20 [神人] 跪求神這位妹子"
write file for "12/20 [正妹] 小倉優香"
write file for "12/20 [正妹] 睡美人"
write file for "12/20 [正妹] 唉呦……不錯哦"
write file for "12/20 Re: [正妹] AKB48 島崎遙香"
write file for "12/20 [正妹] 還可以"
write file for "12/19 [正妹] 好身材"
write file for "12/19 [正妹] Linda"
write file for "12/19 [正妹] 就 阿純"
write file for "12/19 [正妹] 痴的氣息"
write file for "12/19 [神人] 資訊展美女sg"
write file for "12/19 [正妹] 竹東高中"
write file for "12/19 [正妹] 覺得優"
write file for "12/19 [神人] 笑容"
write file for "12/19 [正妹] 銘傳大學"
write file for "12/19 [帥哥] NBA最帥沒有之一:Kobe Bryant"
write file for "12/19 [正妹] 黃芷嫣"
write file for "12/19 [神人] 想請問這位長腿妹"
write file 

write file for "12/07 [正妹] 甜甜的.........."
write file for "12/07 [正妹] 刺青大眼妹"
write file for "12/07 [正妹] 甜美氣質美女"
write file for "12/07 [正妹] 海軍之花~黃靜昀"
write file for "12/07 Re: [正妹] 祈錦鈅 就一張"
write file for "12/07 [正妹] 輕熟姊姊"
write file for "12/07 [正妹] 布莉安娜海德布蘭德"
write file for "12/07 [正妹] 祈錦鈅 就一張"
write file for "12/07 [正妹] 職業籃球啦啦隊"
write file for "12/07 [正妹] Pool 泳池"
write file for "12/07 [神人] 神一位資訊月展場 NEC 的 SG"
write file for "12/07 [正妹] 是記者耶…………"
write file for "12/07 [正妹] 喜歡短髮……"
write file for "12/07 [正妹] 別把螢幕用的白白的…………"
write file for "12/07 [神人] 神一位正妹"
write file for "12/07 [正妹] 笑靨如花"
write file for "12/07 [正妹] 吊帶褲"
write file for "12/07 [正妹] 日本妹子"
write file for "12/07 [正妹] 忍野さら"
write file for "12/07 [神人] 神長榮正妹空姐"
write file for "12/07 [挑戰] 就一張"
write file for "12/07 [正妹] 中正高中"
write file for "12/07 [正妹] 一早來點精神……"
write file for "12/07 [神人] 神一位正妹"
write file for "12/07 [神人] 酷鎷設計model "
write file for "12/07 [正妹] 很愛笑的女孩"
write file for "12/07 [正妹]東找找西找找..........."
write file for "1

write file for "11/28 [正妹] 邂逅兩張"
write file for "11/28 [神人] 高中女生"
write file for "11/28 [正妹] 短髮可愛小圓臉"
write file for "11/28 [正妹] 甜甜氣質"
write file for "11/28 [正妹] 毛衣山丘"
write file for "11/28 [正妹] 黑長直"
write file for "11/28 [正妹] 睡不著"
write file for "11/28 [神人] 韓文課的韓國籍助教"
write file for "11/28 [神人] 星期日高雄FFK扮演冰雪奇緣安娜的COSER"
write file for "11/27 [神人] 影片中的女生"
write file for "11/27 [正妹] 姊"
write file for "11/27 [正妹] 小學生"
write file for "11/27 [正妹] 一個很可愛的女孩"
write file for "11/27 [正妹] 康橋國際學校"
write file for "11/27 [神人] 民視新聞記者"
write file for "11/27 [正妹] Lauren Hanley 美國模特兒 "
write file for "11/27 [正妹] 泳裝IN水池"
write file for "11/27 [正妹] 黑長髮的女孩"
write file for "11/27 [正妹] 大島優子"
write file for "11/27 [正妹] 金子理江"
write file for "11/27 [神人]第六屆 台灣成人博覽會 某SG"
write file for "11/27 [正妹] 豐唇妹子"
write file for "11/27 [神人]  養樂多VS羅德 巨乳球僮"
write file for "11/27 [正妹] 東吳美女"
write file for "11/27 [正妹] 秘書"
write file for "11/27 Re: [正妹] 氣質東京姊姊"
write file for "11/27 [神人] Samsung廣告的妹子"
write file for "11/27 [神人] 這

write file for "11/15 [正妹] 瑞秋 Rachel Q"
write file for "11/15 [正妹]覺得清新........."
write file for "11/15 [正妹] 冷豔但有料的竹竿"
write file for "11/15 [正妹] ㄎㄧㄤ一發"
write file for "11/15 [神人] 外國模特兒"
write file for "11/15 [正妹] 咕硓石圍牆"
write file for "11/15 [正妹] HKT48 指原莉乃"
write file for "11/15 [正妹] 女香理髮廳"
write file for "11/15 [正妹] SKE48 松井珠理奈"
write file for "11/14 [正妹] 晚安幾張"
write file for "11/14 [神人] 可以幫忙神一下這影片的女生嗎？"
write file for "11/14 [神人] 深深"
write file for "11/14 [神人] 修復硬碟時找到的翹臀"
write file for "11/14 [正妹] 到高中協助做健檢的護理師"
write file for "11/14 [正妹] 閒來無事........."
write file for "11/14 [正妹] 電影少女"
write file for "11/14 [神人] 台中全國大飯店-全壽樓服務員"
write file for "11/14 [正妹] 可愛小隻馬"
write file for "11/14 [正妹] 簡短的幾張 清秀美女"
write file for "11/14 [神人] 廣告女主角"
write file for "11/14 [正妹] 十年小池"
write file for "11/14 [正妹] 開飯..............."
write file for "11/14 [正妹] Jessica Hart 澳洲模特兒 177cm"
write file for "11/14 [神人] 據說是直播 的正妹"
write file for "11/14 [正妹] 臉書有共同好友的正妹"
write file for "11/14 [正妹] 微甜空服"
write file f

...

write file for "10/17 [正妹] Elizabeth Hurley 52歲"
write file for "10/17 [正妹] 合法蘿莉"
write file for "10/17 [神人] 大支<王董>MV裡的一位正妹"
write file for "10/17 [神人] 一張老照片"
write file for "10/17 [神人] 朋友的群組裡看到"
write file for "10/17 [神人] 神這個奶妹"
write file for "10/17 [正妹] Celine Farach 19 歲美國辣模、歌手"
write file for "10/17 [正妹] 徐若瑄"
write file for "10/17 [公告] KeMBaWallKer 水桶"
write file for "10/17 Re: [神人] vfashion_korea model"
write file for "10/17 [正妹] 讓你捨不得呼吸"
write file for "10/17 [正妹] 韓國清新正妹"
write file for "10/17 [正妹] 川崎あや"
write file for "10/17 Re: [神人] MotoGP撐傘妹"
write file for "10/17 [正妹] 潤娥"
write file for "10/17 [神人] 長野縣上田高等學校"
write file for "10/16 [神人] 高雄高鐵左營站賣紅豆餅的小女生"
write file for "10/16 [神人] PS4展場SG"
write file for "10/16 [正妹] 趙麗穎 1016生日"
write file for "10/16 [正妹] 輔仁大學正妹"
write file for "10/16 [帥哥] 捕獲可愛男星"
write file for "10/16 [正妹] Leila Lowfire 德國模特兒"
write file for "10/16 [正妹] 長澤茉里奈"
write file for "10/16 [正妹] 抽菸的女生"
write file for "10/16 [正妹] 知性微凶反差天使"
write file for "10/16 [正妹] 黑框眼

write file for "10/06 [正妹] 永尾まりや"
write file for "10/06 [正妹] 短髮"
write file for "10/06 [神人] CHIMEI奇美 model"
write file for "10/06 [神人] 柯文哲嗡嗡包學生妹"
write file for "10/06 [廣告] 吉岡里帆小狐狸為淡定哥星野源偷吃而發怒"
write file for "10/06 [正妹] 吃炒麵"
write file for "10/06 [神人] 7-11 郵差包麻豆"
write file for "10/06 [正妹] Sophie Rose"
write file for "10/06 [正妹] 20幾年前的日雜封面"
write file for "10/06 [正妹] 雀斑少女好身材Annika Boron"
write file for "10/06 [正妹]一張決勝負 我的健身教練們"
write file for "10/06 [正妹] 根本凪(18)"
write file for "10/06 [正妹] 短髮微甜"
write file for "10/06 [正妹] 彰化市第一正妹"
write file for "10/05 [神人] 韓國妹直播跳舞求神"
write file for "10/05 [公告] 板規修訂 - 意淫文字"
write file for "10/05 [神人] 大安站旁邊壽司店正妹店員"
write file for "10/05 [正妹] 東山高中"
write file for "10/05 [正妹] 短髮空姐"
write file for "10/05 [公告] alan23273850 水桶"
write file for "10/05 [神人] 網路正妹"
write file for "10/05 [神人] 7-11 獎三遍 中的 女同事"
write file for "10/05 [正妹] 超級陣容"
write file for "10/05 [正妹] 黑鳥"
write file for "10/05 [正妹] 我私藏的松岡茉優"
write file for "10/05 [神人] 網路正妹"
write file for "10/05 

write file for "9/24 [帥哥] 跪求這位Ikea新莊店帥哥推車手FB"
write file for "9/24 [正妹] 奶特(微多)"
write file for "9/24 [正妹] 中山女中"
write file for "9/24 [正妹] 微甜氣質 "
write file for "9/24 [正妹] 不錯"
write file for "9/24 [正妹] Lindsay Brewer 美國科羅拉多人業餘賽車手"
write file for "9/24 [正妹] 教練我想打羽球"
write file for "9/24 [正妹] 可愛小隻馬"
write file for "9/24 [正妹] 有點兒輕熟～"
write file for "9/24 [正妹] 圓臉可愛"
write file for "9/24 [正妹] 東京電玩展：魔物獵人受付娘Coser"
write file for "9/24 [神人] 請問報導裡的豐滿妹是誰"
write file for "9/24 [神人]who is she?"
write file for "9/23 [正妹] 麻醉風暴 整形外科醫師 娜娜"
write file for "9/23 [正妹] 新店高中二連發"
write file for "9/23 [正妹] MAMAMOO 丁輝人"
write file for "9/23 [正妹] Ipair直播正妹 台大素顏妹"
write file for "9/23 [正妹] 內衣模特兒"
write file for "9/23 [正妹] 想當那隻手機"
write file for "9/23 [正妹] 寺田蘭世 生日快樂～～～"
write file for "9/23 [正妹] 妙妙"
write file for "9/23 [神人] toyota altis x 的車模"
write file for "9/23 [正妹] 馬尾姐姐 好翹~"
write file for "9/23 [正妹] 手指愛心"
write file for "9/23 [正妹] 圓臉女孩"
write file for "9/23 [神人] 亞太 MODEL"
write file for "9/22 [神人] Honda fit

write file for "9/10 [正妹] 阿純水水"
write file for "9/10 [正妹] 這樣的女生可愛嗎?"
write file for "9/10 [神人] 清秀正妹"
write file for "9/10 [神人] 求神一位model"
write file for "9/10 [正妹] Milla"
write file for "9/10 [正妹] 小鹿 亂撞"
write file for "9/10 [神人] 9/7思x爾內衣秀的模特兒"
write file for "9/10 [正妹] TWICE TZUYU"
write file for "9/10 [正妹]  一張一影片 360度 石頭"
write file for "9/09 [正妹] 金村美玖"
write file for "9/09 [正妹] Mita"
write file for "9/09 [神人] 統一豆漿廣告裡面的女生"
write file for "9/09 [神人] Sony 手機廣告"
write file for "9/09 [神人]小米 A1的SG"
write file for "9/09 [正妹] 捕獲女明星自然美"
write file for "9/09 [神人] 採訪王世堅剃頭的女記者"
write file for "9/09 [正妹] 甄嬛傳 溫宜公主（紀姿含）"
write file for "9/09 Re: [神人] 熊本內餡"
write file for "9/09 [正妹] Autumn Holley 美國模特兒"
write file for "9/09 [廣告] 國光幫女神 小淨 第二發"
write file for "9/09 [正妹] 有點凸出的 牙齒"
write file for "9/09 [神人] 行政院"
write file for "9/09 [正妹] 刺青的萌妹"
write file for "9/09 [正妹] 洋妞"
write file for "9/09 [神人] 右邊的兇妹"
write file for "9/09 [正妹] C92 愛醬"
write file for "9/09 [神人] 西門町 永富冰淇淋的男店員？"
write file for "9/09 

TypeError: string indices must be integers